In [ ]:
# Install poincare in environment
! pip install poincare

# Simulation in poincare
Poincare and SimBio include a number of different options for simulation, including changing solvers and backends, searching for steady states and limit cycles with parameter sweeps and interactive simulation.

### Using different solvers and backends
We can change the method used for simulating the system by passing it to the `solve` method. To use Runge-Kutta  of order 4 (5) instead of the default LSODA (SciPy's implementation of Adams/BDF method):

In [21]:
from poincare import (
    System,
    Variable,
    Derivative,
    Constant,
    Parameter,
    Simulator,
    initial,
    assign,
)
import numpy as np


# Define a System
class DampedOscillator(System):
    x: Variable = initial(default=1)
    v: Derivative = x.derive(initial=0)
    k: Constant = assign(default=1, constant=True)
    eq = v.derive() << -k * x - 0.1 * v

In [22]:
from poincare import solvers

result = Simulator(DampedOscillator).solve(
    solver=solvers.RK45(),
    save_at=range(3),
)
result

,x,v
time,,
0,1.000000,0.000000
1,0.554945,-0.800974
2,-0.333405,-0.824557


Currently the implemented methods are: Asams/BDF (LSODA), Runge-Kutta of orders 3(2) (RK32), 4(5) (RK45) and 8 (DOP853), Runge-Kutta method of Radau IIA family of order 5 (Radau), and BDF (BDF). All oft them are wrappers to SciPy's implementation  of the method, see [SciPy's documentation](https://docs.scipy.org/doc/scipy/reference/integrate.html) for more details.


Poincare also allows the use of diffent backends to compile the systems to, inculuding [NumPy](https://numpy.org/) (by default) [Numba](https://numba.pydata.org/) and [JAX](https://docs.jax.dev/en/latest/):

In [23]:
result = Simulator(DampedOscillator, backend="numba").solve(save_at=range(3))
result

,x,v
time,,
0,1.000000,0.000000
1,0.555134,-0.801076
2,-0.333635,-0.825498


### Interactive simulators
We can use the `interact` method create interactive simulations to vary parameters or initial conditions:



In [39]:
result = Simulator(DampedOscillator).interact(
    save_at=np.linspace(0, 10, 100),
    # values takes a tuple with (start, stop, step)
    values={DampedOscillator.k: (0, 10, 0.1)},
)

interactive(children=(FloatSlider(value=1.0, description='k', max=10.0), Output()), _dom_classes=('widget-inte…